In [1]:
#Imports
import math
import time
from tkinter import * 
from tkinter import ttk
from tkinter.filedialog import askopenfile 
from PIL import ImageTk,Image

In [ ]:
class BLAST:
  """
  Local Alignment of a query DNA Sequences with a database sequence
  """
  def __init__(self, l_mer_length = 11 , query_string = "" , sequence_string = "", hssp_score = 8):
    self.start_time = time.time()
    query = query_string.upper().replace("N" , "")
    dna = sequence_string.upper().replace("N" , "")
    self.l = l_mer_length        # Default word length = 11
    self.query = query
    self.q = len(query)
    self.dna_sequence = dna
    self.n = len(dna)
    self.score_params = { "match":1, "gap_insertion": -1 , "mismatch": -1 , "gap_extension": -1 }   
    self.extension_threshold =  math.ceil(hssp_score*3/4) # If the score drops below 1 while extension we stop extending
    self.E_threshold = 0.005
    self.HSSP_score = hssp_score # high scoring sequence pairs score
    self.nuc_dict = {"A":0, "T":3, "C":1 , "G":2, }  # Using numerical representation of nucleotides
    self.sequence_words = {}     # { key_of_the_word: [list of starting positions] }
    self.query_words = []        # List of query l-mers and their mutation with ( with score > HSSP Score )
    self.matched = {}            # { starting_pos_in_query : [list of starting pos in DNA sequence ]}
    self.local_alignments = []   # List of [ [ query_start_pos, query_end_pos, seq_start_pos, seq_end_pos]]
    self.param_lambda = 0        # parameter for statistical significance
    self.param_k  = 0
    
    # Lists for graphs 
    self.pos_list = []  #starting positions
    self.E_list = []    #corresponding E value
    self.prob_list = [] #corresponding probabilities

    self.finding_sequence_words()
    self.finding_query_words()
    self.seeding()
    self.extension()
    self.calculate_params()
    self.display()
    self.end_time = time.time()

  def calculate_params(self):
    """
    Calculating constants lambda and K , for Karlin Atschul equation. These constants are dependent on the probability of nucleotide in query and DNA Sequence and 
    the scoring scheme. 
    """
    freq_nuc = {}
    nucs="ATCG"
    for nuc in nucs:
      freq_nuc[nuc]= [ self.query.count(nuc)/self.q , self.dna_sequence.count(nuc)/self.n]
    q = 0 #negative S(i,j) 
    p = 0 #positive S(i,j)
    for i in nucs:
      p  += freq_nuc[i][0]*freq_nuc[i][1]
      for j in nucs:
        if j != i :
          q  += freq_nuc[i][0]*freq_nuc[j][1]
    
    self.param_lambda  = math.log(p/q)
    self.param_k = ((q-p)**2)/q
    
  
  def probability(self, i):
  
    score = self.score_words(self.query[i[0]: i[1]+1] , self.dna_sequence[i[2]: i[3]+1 ], string=True )
    E = self.param_k*self.q*self.n*math.exp(self.param_lambda*score)
    prob = 1 - math.exp(-E)
    return (prob, E , score)


  def key(self, list_of_seq):
    """
    Converts the string representation of a k-mer to encoded key using the numerical representation
      # e.g. TGAC = 3*(4**3) +  2*(4**2) + 0*(4**0) + 1*(4**1)
    """
    list_of_keys = []
    for seq in list_of_seq:
      key = 0 
      for i in range(len(seq)):
        key += self.nuc_dict[seq[i].upper()]*(4**(len(seq)-i-1))
      list_of_keys.append(key)
    return list_of_keys

  def finding_sequence_words(self):
    """
    Finding all the l-mers of the given DNA sequence and hashing their key with the starting position.
    """
    
    for i in range(self.n - self.l + 1 ):
      key = self.key([self.dna_sequence[i:self.l+i]])[0]
      if key in self.sequence_words:
        self.sequence_words[key].append(i)
      else:
        self.sequence_words[key] = [i]

  def finding_query_words(self):
    """
    Finding the all the l-mers and their mutation with ( with score > HSSP Score ) of the given query strings
    """

    for i in range(self.q - self.l + 1):  
      sequences = self.find_mutations(self.query[ i : self.l+i ].upper())
      self.query_words.append(self.key(list(set(sequences))))

  def find_mutations(self, seq):
    """
    Finds all the possible sequences with 0, 1, 2 mutations in the given sequence
    """
    list_of_mutations = [seq]
    nucleotide = [ 'A', 'T', 'G', 'C'] 
    for i in range(len(seq)):
      for  t in nucleotide:
        if(t != seq[i]):
          mut = seq
          mut = mut[:i] + t + mut[i+1:]
          list_of_mutations.append(mut)
          for j in range(i+1, len(seq)):
            for t in nucleotide:
              mut2 = mut
              if(t != mut[j]):
                mut2 = mut2[:j] + t + mut2[j+1:]
                list_of_mutations.append(mut2)
    return list_of_mutations

  def seeding(self):
    """
    Finding matches between sequence words and HSSP from query string 
    """
    for pos in range(len(self.query_words)):
      for seq in self.query_words[pos]:
        if seq in self.sequence_words:
          if pos in self.matched:
            self.matched[pos].extend(self.sequence_words[seq])
          else:
            self.matched[pos] = self.sequence_words[seq]
  
  def extension(self):
    """
    Extending the seeded matches using Smith Waterman Algorithm ( ungapped )
    """
    for query_pos in self.matched:
      for seq_pos in self.matched[query_pos]:
        score = self.score_words(query_pos, seq_pos)
        i = query_pos
        j = seq_pos
        s = score
        while ( i > 0 and j > 0 and s >= self.extension_threshold ):
          if self.query[i-1] == self.dna_sequence[j-1]:
            s+= self.score_params["match"]
          else:
            s+= self.score_params["mismatch"]
          i-=1
          j-=1
        query_start = i
        seq_start = j
        i = query_pos + self.l - 1
        j = seq_pos + self.l - 1
        s = score
        while ( i < (self.q-1) and j < (self.n -1) and s >= self.extension_threshold ):
          if self.query[i+1] == self.dna_sequence[j+1]:
            s+= self.score_params["match"]
          else:
            s+= self.score_params["mismatch"]
          i+=1
          j+=1
        query_stop = i
        seq_stop = j
        alignment = [query_start, query_stop, seq_start, seq_stop]
        if alignment not in self.local_alignments:
          self.local_alignments.append(alignment)
  
  def score_words(self, query_pos, seq_pos, string=False):
    """
    Return score for a pair of words
    """
    score =  0
    if string:
      for i in range(len(query_pos)):
        if query_pos[i] == seq_pos[i]:
          score+= self.score_params["match"]
        # else:
        #   score+= self.score_params["mismatch"]
      score = (score*100)/self.q
    else:   
      for i in range(self.l):
        if query_pos+i >= self.q or seq_pos+i >= self.n:
          break
        elif self.query[query_pos+i] == self.dna_sequence[seq_pos+i]:
          score+= self.score_params["match"]
        else:
          score+= self.score_params["mismatch"]
    return score
  
  def display(self):
    """
    Displays local alignment
    """
    if len(self.local_alignments)==0 :
      print ( "No alignment found")
    count=0
    for alignment in self.local_alignments:
      prob, E , score  = self.probability(alignment)
      self.pos_list.append(alignment[2])
      self.E_list.append(E)
      self.prob_list.append(prob)
      if E<= self.E_threshold:
        count=count+1
        print("Probality:", prob)
        print("E Value:  ", E)
        print("Score: ", score)
        print("Query Cover: ", round((alignment[1]-alignment[0]+1)*100/self.q , 2), "%")
        q_pos = alignment[0]
        seq_pos = alignment[2]
        s= "DNA Sequence(Start Position="+ str(seq_pos)+"): " 
        k = len(s)*" "
        ss = "\nQuery(Start Position="+ str(q_pos)+"): "
        kk = (len(s)-len(ss))*" "
        print( s + self.dna_sequence[alignment[2]: alignment[3]+1 ])
        print(k, end ="")
        a=0
        
        cnt = alignment[3] - alignment[2] 
        while ( a <= cnt):
          if self.query[q_pos+a] == self.dna_sequence[seq_pos+a]:
            print("|" , end ="")
          else: 
            print(" " , end ="")
          a+=1

        print( ss +kk+" "+ self.query[alignment[0]: alignment[1]+1 ], end="\n\n\n")
    print("Total Number of Hits Found: ",count)   
  
        

In [ ]:
def main():
  #Sequence string fasta
  x=open("/content/ex.fasta")
  a=x.read()
  ind=a.find("\n")
  a=a[ind:]
  seque=""
  for i in range(len(a)):
    if a[i]!="\n":
      seque=seque+str(a[i])
  print("Printing length of Subject sequence: ", len(seque) )

  #Query string fasta
  x1=open("/content/ex2.fasta")
  a1=x1.read()
  ind1=a1.find("\n")
  a1=a1[ind1:]
  quer=""
  for i in range(len(a1)):
    if a1[i]!="\n":
      quer=quer+str(a1[i])
  print("Printing length of Query sequence: ", len(quer))

  blast = BLAST( query_string = quer , sequence_string = seque )
  print("Time Required: ",blast.end_time-blast.start_time)

main()

Printing length of Subject sequence 29903
Printing length of Query sequence 198
Probality: 0.002827025456615062
E Value:   0.0028310290403503148
Score:  21.27659574468085
Query Cover:  40.96 %
DNA Sequence(Start Position=11604): GCTAGTTTATTGTTTCTTAGGCTATTTTTGTACTTGTTACTTTGGCCTCTTTTGTTTACTCAACCGCTACTTTAGAC
                                      |||||| || ||| ||    | ||||| |  || ||  |||    | |||| |||  |       |  |||    
Query(Start Position=70):           AGTAGTTTCTTCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT


Probality: 0.0010359593143238754
E Value:   0.0010364962910637922
Score:  22.340425531914892
Query Cover:  43.09 %
DNA Sequence(Start Position=9572): GTGTTTATTCTGTTATTTACTTGTACTTGACATTTTATCTTACTAATGATGTTTCTTTTTTAGCACATATTCAGTGGATGG
                                     |||| |||| || |||  || |  ||    |||| | ||  |  |  | ||| ||||||      | ||   |   |  
Query(Start Position=72):          TAGTTTCTTCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT